In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import jieba 
import re
import sklearn
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


In [7]:
from matplotlib.font_manager import FontProperties
myfont=FontProperties(fname=r'C:\Windows\Fonts\simhei.ttf',size=14)
sns.set(font=myfont.get_name())

df_1=pd.read_csv(r'C:/git/a/b/p3-1.csv',error_bad_lines=False,\
                 header=None,names=['销量','价格','厂家','d','e','f','g'])

In [9]:
del df_1['d']
df_1['内容'] = df_1['e']
del df_1['e']
xl = df_1['销量']
def reduce_xl(x):
    x = re.sub(' 去看二手',' ',x)
    x = re.match(r'(.*)(\d)(.*)(\+)',x)
    if x: 
        x = re.sub(' ','',x.group(0))
        if(re.match(r'.*万',x[:-1])):
            x=re.match(r'.*万',x[:-1]).group(0)[:-1]
            x = float(x)*10000
        else:
            x = float(x[:-1])
        return x
    else:
        return 0
xl = xl.apply(lambda x:  reduce_xl(x))

In [10]:
jg = df_1['价格']
def reduce_jg(x):
    if (x!=0):
        x = re.sub(' ￥','',x)
        x = re.sub('.00','',x)
    else:
        x = 0
    return float(x)
jg = jg.apply(lambda x:  reduce_jg(x))

In [13]:
cj = df_1['厂家']
def reduce_cj(x):
    x=jieba.lcut(x[1:])
    if x:
        return x[0]
    else:
        return ' '
cj = cj.apply(lambda x:  reduce_cj(x))

In [14]:
nr = df_1['内容']
def reduce_nr(x):
    x = re.sub(r' +',' ',x)
    x = re.sub(r'【.+】','',x)
    x = re.split(r' ',x[1:])
    g5 = ''
    hz = ''
    mah = ''
    w = ''
    gb = ''
    for i in x:
        G5 = re.search(r'5G',i)
        if G5:
            if g5 == '':
                g5 = i
        Hz = re.search(r'Hz',i)
        if Hz:
            hz = i
        mAh = re.search(r'mAh',i)
        if mAh :
            mah = i
        W = re.search(r'\d+W',i)
        if W :
            w = i
        GB = re.search(r'\d+GB',i)
        if GB :
            gb = i
    return [g5,hz,mah,w,gb]
nr_1 = nr.apply(lambda x:  reduce_nr(x))
df_2 = nr_1.to_frame()
df_2['g5'] = df_2['内容'].apply(lambda x: x[0]) 
df_2['hz'] = df_2['内容'].apply(lambda x: x[1]) 
df_2['mah'] = df_2['内容'].apply(lambda x: x[2]) 
df_2['w'] = df_2['内容'].apply(lambda x: x[3]) 
df_2['gb'] = df_2['内容'].apply(lambda x: x[4]) 
def reduce_df_21(x):
    if x == '':
        x = 0
    else:
        x = 1
    return x
s_g5 = df_2['g5'].apply(lambda x: reduce_df_21(x))
def reduce_df_22(x):
    x = re.search(r'(\d+)Hz',x)
    if x:
        x = float(x.group(0)[:-2])
    else:
        x = 0
    return x
s_hz = df_2['hz'].apply(lambda x: reduce_df_22(x))
def reduce_df_23(x):
    x = re.search(r'(\d+)mAh',x)
    if x:
        x = float(x.group(0)[:-3])
    else:
        x = 0
    return x
s_mah = df_2['mah'].apply(lambda x: reduce_df_23(x))
def reduce_df_24(x):
    x = re.search(r'(\d+)W',x)
    if x:
        x = float(x.group(0)[:-1])
    else:
        x = 0
    return x
s_w = df_2['w'].apply(lambda x: reduce_df_24(x))
def reduce_df_25(x):
    x = re.search(r'(\d+)',x)
    if x:
        x = float(x.group(0))
        if x>32:
            x = 0
    else:
        x = 0
    return x
s_gb = df_2['gb'].apply(lambda x: reduce_df_25(x))
data = pd.DataFrame({'gb':s_gb,'g5':s_g5,'hz':s_hz,'mah':s_mah,\
              'w':s_w,'jg':jg,'cj':cj,'xl':xl})